# Object Detection - Single Shot detection
## 실습 : 자율주행 자동차를 위한 SSD7 학습
* SSD7 - SSD300의 7층짜리 축소버전

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/com_vision_study/ssd')

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TerminateOnNaN, CSVLogger
from tensorflow.keras import backend as K
from keras.models import load_model
from math import ceil
import numpy as np
import tensorflow.python.keras.engine

import matplotlib.pyplot as plt
from tensorflow.keras.layers import Layer, InputSpec

from models.keras_ssd7 import build_model
from keras_loss_function.keras_ssd_loss import SSDLoss
from keras_layers.keras_layer_AnchorBoxes import AnchorBoxes
from keras_layers.keras_layer_DecodeDetections import DecodeDetections
from keras_layers.keras_layer_DecodeDetectionsFast import DecodeDetectionsFast

from ssd_encoder_decoder.ssd_input_encoder import SSDInputEncoder
from ssd_encoder_decoder.ssd_output_decoder import decode_detections, decode_detections_fast

from data_generator.object_detection_2d_data_generator import DataGenerator
from data_generator.object_detection_2d_misc_utils import apply_inverse_transforms
from data_generator.data_augmentation_chain_variable_input_size import DataAugmentationVariableInputSize
from data_generator.data_augmentation_chain_constant_input_size import DataAugmentationConstantInputSize
from data_generator.data_augmentation_chain_original_ssd import SSDDataAugmentation

In [ ]:
# Parameters Setting

# 입력 이미지 (너비, 높이, 채널)
img_height = 300 
img_width = 480 
img_channels = 3 

# Set this to your preference (maybe `None`). The current settings transform the input pixel values to the interval `[-1,1]`.
intensity_mean = 127.5 
intensity_range = 127.5 

# Number of classes in our dataset
# 자동차, 트럭, 보행자, 자전거, 신호등
n_classes = 5 

# 앵커박스 배율 리스트 --> 이 리스트는 min_scale과 max_scale인수값을 상속받음
scales = [0.08, 0.16, 0.32, 0.64, 0.96]


aspect_ratios = [0.5, 1.0, 2.0] # 앵커박스의 종횡비
two_boxes_for_ar1 = True # 종횡비가 1인 앵커박스를 2개 생성할지 여부
steps = None # 앵커박스 격자의 이미지 내 시작 위치를 직접 설정할 경우에는 권장하지 않는다.
offsets = None # 앵커박스 격자의 간격을 직접 설정할 경우는 권장하지 않는다
clip_boxes = False # 앵커 밖으로 빠져나가는 앵커박스를 절단할지 여부
variances = [1.0, 1.0, 1.0, 1.0] # 인코딩된 목표 좌표의 배율을 조정할 때 적용할 분산값이 기재된 리스트
normalize_coords = True # 이미지 크기에 대한 상대좌표 사용 여부

In [ ]:
# 실습 : Udacity 이미지 데이터셋 사용 --> 이미지 크기 480X300
#       전체 이미지 22000장

K.clear_session()

model = build_model(image_size=(img_height, img_width, img_channels),
                    n_classes=n_classes,
                    mode='training',
                    l2_regularization=0.0005,
                    scales=scales,
                    aspect_ratios_global=aspect_ratios,
                    aspect_ratios_per_layer=None,
                    two_boxes_for_ar1=two_boxes_for_ar1,
                    steps=steps,
                    offsets=offsets,
                    clip_boxes=clip_boxes,
                    variances=variances,
                    normalize_coords=normalize_coords,
                    subtract_mean=intensity_mean,
                    divide_by_stddev=intensity_range
                    )

adam = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
ssd_loss = SSDLoss(neg_pos_ratio=3, alpha=1.0)
model.compile(optimizer=adam, loss=ssd_loss.compute_loss)

In [ ]:
# Make Dataset

train_dataset = DataGenerator(load_images_into_memory=False, hdf5_dataset_path=None)
valid_dataset = DataGenerator(load_images_into_memory=False, hdf_5_dataset_path=None)

images_dir = '/content/drive/MyDrive/com_vision_study/ssd/udacity_driving_datasets'

train_labels_filename = '/content/drive/MyDrive/com_vision_study/ssd/udacity_driving_datasets/labels_train.csv'
valid_labels_filename = '/content/drive/MyDrive/com_vision_study/ssd/udacity_driving_datasets/labels_val.csv'

train_dataset.parse_csv(images_dir= images_dir, 
                        labels_filename=train_labels_filename, 
                        input_format = ['image_name', 'xmin', 'xmax', 'ymin', 'ymax', 'class_id'],
                        include_classes='all')

valid_dataset.parse_csv(images_dir=images_dir, 
                        labels_filename=train_labels_filename,
                        input_format = ['image_name', 'xmin', 'xmax', 'ymin', 'ymax', 'class_id'],
                        include_classes='all')

train_dataset_size = train_dataset.get_dataset_size()
valid_dataset_size = valid_dataset.get_dataset_size()

print(train_dataset_size)
print(valid_dataset_size)

In [ ]:
batch_size = 16

data_augmentation_chain = DataAugmentationConstantInputSize(
                              random_brightness = (-48, 48, 0.5),
                              random_contrast = (0.5, 1.8, 0.5),
                              random_saturation = (0.5, 1.8, 0.5),
                              random_hue = (18, 0.5),
                              random_flip=0.5,
                              random_scale=(0.03, 0.5),
                              n_trials_max = 3,
                              clip_boxes = True,
                              overlap_criterion='area',
                              bounds_box_filter = (0.3, 1.0),
                              n_boxes_min=1,
                              background=(0,0,0)
                          )

predictor_sizes = [model.get_layer('classes4').output_shape[1:3],
                   model.get_layer('classes5').output_shape[1:3],
                   model.get_layer('clasess6').output_shape[1:3],
                   model.get_layer('classes7').output_shape[1:3]]

ssd_input_encoder = SSDInputEncoder(img_height=img_height, 
                                    img_width=img_width, n_classes=n_classes,
                                    predictor_sizes=predictor_sizes,
                                    scales=scales,
                                    aspect_ratios_global=aspect_ratios,
                                    two_boxes_for_ar1=two_boxes_for_ar1,
                                    steps=steps,
                                    offsets=offsets,
                                    clip_boxes=clip_boxes,
                                    variances=variances,
                                    matching_type='multi',
                                    pos_iou_threshold=0.5,
                                    neg_iou_limit=0.3,
                                    normalize_coords=normalize_coords)



# keras fit_generator()함수에 전달할 제너레이터 객체 생성
train_generator = train_dataset.generate(batch_size=batch_size, shuffle=True,
                                         transformations=[
                                                          data_augmentation_chain],
                                         label_encoder=ssd_input_encoder,
                                         returns={'processed_images', 'encoded_labels'},
                                         keep_images_without_gt=False)

valid_generator = valid_dataset.generate(batch_size=batch_size, shuffle=False,
                                         transformations=[
                                                          data_augmentation_chain],
                                         label_encoder=ssd_input_encoder,
                                         returns={'processed_images', 'encoded_labels'},
                                         keep_images_without_gt=False)

In [ ]:
cp = ModelCheckpoint('ssd7_weights.h5', monitor='val_loss', verbose=1, save_best_only=True,
                     save_weights_only=False, mode='auto', period=1)

csv_logger = CSVLogger(filname='ssd7_training_log.csv',
                       separator=',', append=True)
es = EarlyStopping(monito='val_loss', min_delta=0.0, patience=10,
                   verbose=1)
rp = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=8, verbose=1,
                       epsilon=0.001, cooldown=0, min_lr=0.00001)

callbacks = [cp, csv_logger, es, rp]

In [ ]:
initial_epoch=0
final_epoch=20
steps_per_epoch=1000  # 1 epoch 당 가중치 1000번 수정

history = model.fit_generator(generator=train_generator, steps_per_epoch=steps_per_epoch,
                              epochs=final_epoch, callbacks=callbacks, validation_data=valid_generator,
                              validation_steps=ceil(valid_dataset_size/batch_size),
                              initial_epoch=initial_epoch)

In [ ]:
plt.figure(figsize=(20,12))
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.legend(loc='upper_right', prop={'size' : 24})